In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [4]:
import pandas as pd
train = pd.read_csv('/content/train_LZdllcl.csv')
test = pd.read_csv('/content/test_2umaH9m.csv')

In [5]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)
train.head()
train.info()
train.describe()
train.isnull().sum()


Train shape: (54808, 14)
Test shape: (23490, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(

,0
employee_id,0
department,0
region,0
education,2409
gender,0
recruitment_channel,0
no_of_trainings,0
age,0
previous_year_rating,4124
length_of_service,0


In [10]:
#Impute missing values
from sklearn.impute import SimpleImputer

edu_imputer = SimpleImputer(strategy='most_frequent')
train['education'] = edu_imputer.fit_transform(train[['education']]).flatten()
test['education'] = edu_imputer.transform(test[['education']]).flatten()

rating_imputer = SimpleImputer(strategy='median')
train['previous_year_rating'] = rating_imputer.fit_transform(train[['previous_year_rating']]).flatten()
test['previous_year_rating'] = rating_imputer.transform(test[['previous_year_rating']]).flatten()

In [11]:
train.isnull().sum()

,0
employee_id,0
department,0
region,0
education,0
gender,0
recruitment_channel,0
no_of_trainings,0
age,0
previous_year_rating,0
length_of_service,0


In [12]:
test.isnull().sum()

,0
employee_id,0
department,0
region,0
education,0
gender,0
recruitment_channel,0
no_of_trainings,0
age,0
previous_year_rating,0
length_of_service,0


In [15]:
#label encoding performed through feature by fror loop and select datatype object/string for lb
le = LabelEncoder()
for col in train.select_dtypes(include='object').columns:
    if col != 'target':
        train[col] = le.fit_transform(train[col])
        test[col] = le.transform(test[col])


In [ ]:
train

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,7,31,2,0,2,1,35,5.0,8,1,0,49,0
1,65141,4,14,0,1,0,1,30,5.0,4,0,0,60,0
2,7513,7,10,0,1,2,1,34,3.0,7,0,0,50,0
3,2542,7,15,0,1,0,2,39,1.0,10,0,0,50,0
4,48945,8,18,0,1,0,1,45,3.0,2,0,0,73,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54803,3030,8,5,0,1,2,1,48,3.0,17,0,0,78,0
54804,74592,4,19,2,0,0,1,37,2.0,6,0,0,56,0
54805,13918,0,0,0,1,0,1,27,5.0,3,1,0,79,0
54806,13614,7,33,0,1,2,1,29,1.0,2,0,0,45,0


In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = train.drop('is_promoted', axis=1)
y_train = train['is_promoted']
X_test = test.copy()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (54808, 13)
Shape of y_train: (54808,)
Shape of X_test: (23490, 13)


In [17]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

models = {
    "RandomForest": RandomForestClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(eval_metric='logloss')
}

from sklearn.metrics import accuracy_score, classification_report

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_train_scaled)
    print(f" {name} Accuracy:", accuracy_score(y_train, y_pred))
    print(classification_report(y_train, y_pred))


 RandomForest Accuracy: 0.9999817544883959
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     50140
           1       1.00      1.00      1.00      4668

    accuracy                           1.00     54808
   macro avg       1.00      1.00      1.00     54808
weighted avg       1.00      1.00      1.00     54808

 GradientBoosting Accuracy: 0.9384943803824259
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     50140
           1       0.96      0.29      0.45      4668

    accuracy                           0.94     54808
   macro avg       0.95      0.64      0.71     54808
weighted avg       0.94      0.94      0.92     54808

 XGBoost Accuracy: 0.9521055320391184
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     50140
           1       0.98      0.44      0.61      4668

    accuracy                           0.95     54808


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Define the range of n_estimators to test
n_estimators_values = [50, 100, 150]

# Dictionary to store performance metrics for each n_estimators value
rf_performance = {}

# Function to extract metrics from classification report (using weighted average)
def extract_metrics(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)
    accuracy = report['accuracy']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1 = report['weighted avg']['f1-score']
    return accuracy, precision, recall, f1

# Iterate through different n_estimators values
for n in n_estimators_values:
    print(f"Training and evaluating Random Forest with n_estimators = {n}")
    # Initialize the Random Forest model
    rf_model = RandomForestClassifier(n_estimators=n, random_state=42)

    # Train the model
    rf_model.fit(X_train, y_train)

    # Make predictions on the training set for evaluation
    y_pred_rf_tuned = rf_model.predict(X_train)

    # Evaluate the model and store the metrics
    accuracy, precision, recall, f1 = extract_metrics(y_train, y_pred_rf_tuned)
    rf_performance[n] = {
        'Accuracy': accuracy,
        'Precision (Weighted Avg)': precision,
        'Recall (Weighted Avg)': recall,
        'F1-Score (Weighted Avg)': f1
    }


print("\nFinished Random Forest hyperparameter tuning.")

Training and evaluating Random Forest with n_estimators = 50
Training and evaluating Random Forest with n_estimators = 100
Training and evaluating Random Forest with n_estimators = 150

Finished Random Forest hyperparameter tuning.


In [31]:
# Create a pandas DataFrame from the rf_performance dictionary
rf_performance_df = pd.DataFrame.from_dict(rf_performance, orient='index')

# Rename the index to 'n_estimators'
rf_performance_df.index.name = 'n_estimators'

# Sort the DataFrame by Accuracy
rf_performance_df = rf_performance_df.sort_values(by='Accuracy', ascending=False)

# Display the table
print("Random Forest Model Performance with Different n_estimators:")
display(rf_performance_df)

Random Forest Model Performance with Different n_estimators:


,Accuracy,Precision (Weighted Avg),Recall (Weighted Avg),F1-Score (Weighted Avg)
n_estimators,,,,
100,1.000000,1.000000,1.000000,1.000000
150,1.000000,1.000000,1.000000,1.000000
50,0.999617,0.999617,0.999617,0.999616


In [34]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# Define the range of n_estimators to test
n_estimators_values = [50, 100, 150]

# Performance dictionaries
rf_performance = {}
gb_performance = {}
xgb_performance = {}

# Function to extract metrics from classification report (using weighted average)
def extract_metrics(y_true, y_pred):
    report = classification_report(y_true, y_pred, output_dict=True)
    accuracy = report['accuracy']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1 = report['weighted avg']['f1-score']
    return accuracy, precision, recall, f1

### RANDOM FOREST
print("🔍 Random Forest Hyperparameter Tuning")
rf_predictions = {}
for n in n_estimators_values:
    print(f"Training and predicting with Random Forest (n_estimators = {n})")
    rf_model = RandomForestClassifier(n_estimators=n, random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_test)
    rf_predictions[n] = y_pred_rf


### GRADIENT BOOSTING
print("\n🔍 Gradient Boosting Hyperparameter Tuning")
gb_predictions = {}
for n in n_estimators_values:
    print(f"Training and predicting with Gradient Boosting (n_estimators = {n})")
    gb_model = GradientBoostingClassifier(n_estimators=n, random_state=42)
    gb_model.fit(X_train, y_train)
    y_pred_gb = gb_model.predict(X_test)
    gb_predictions[n] = y_pred_gb

### XGBOOST
print("\n🔍 XGBoost Hyperparameter Tuning")
xgb_predictions = {}
for n in n_estimators_values:
    print(f"Training and predicting with XGBoost (n_estimators = {n})")
    xgb_model = XGBClassifier(n_estimators=n, eval_metric='logloss', random_state=42)
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_test)
    xgb_predictions[n] = y_pred_xgb

# Print summary
print("\n✅ Finished Hyperparameter Tuning and Prediction")

# Optionally display each result clearly
print("\n=== Random Forest Predictions (first 5) ===")
for n, preds in rf_predictions.items():
    print(f"n_estimators = {n} -> {preds[:5]}")

print("\n=== Gradient Boosting Predictions (first 5) ===")
for n, preds in gb_predictions.items():
    print(f"n_estimators = {n} -> {preds[:5]}")

print("\n=== XGBoost Predictions (first 5) ===")
for n, preds in xgb_predictions.items():
    print(f"n_estimators = {n} -> {preds[:5]}")

🔍 Random Forest Hyperparameter Tuning
Training and predicting with Random Forest (n_estimators = 50)
Training and predicting with Random Forest (n_estimators = 100)
Training and predicting with Random Forest (n_estimators = 150)

🔍 Gradient Boosting Hyperparameter Tuning
Training and predicting with Gradient Boosting (n_estimators = 50)
Training and predicting with Gradient Boosting (n_estimators = 100)
Training and predicting with Gradient Boosting (n_estimators = 150)

🔍 XGBoost Hyperparameter Tuning
Training and predicting with XGBoost (n_estimators = 50)
Training and predicting with XGBoost (n_estimators = 100)
Training and predicting with XGBoost (n_estimators = 150)

✅ Finished Hyperparameter Tuning and Prediction

=== Random Forest Predictions (first 5) ===
n_estimators = 50 -> [0 0 0 0 0]
n_estimators = 100 -> [0 0 0 0 0]
n_estimators = 150 -> [0 0 0 0 0]

=== Gradient Boosting Predictions (first 5) ===
n_estimators = 50 -> [0 0 0 0 0]
n_estimators = 100 -> [0 0 0 0 0]
n_estima

In [72]:
# Initialize the best model with best hyperparameter
best_model = XGBClassifier(n_estimators=150, eval_metric='logloss', random_state=42)

# Train the best model
best_model.fit(X_train_scaled, y_train)

# Predict on the scaled test data
final_preds = best_model.predict(X_test_scaled)


In [73]:
sample = pd.read_csv('/sample_submission_M0L0uXE (1).csv')

In [74]:
submission = sample.copy()
submission['is_promoted'] = final_preds
submission.to_csv('final_submission.csv', index=False)


In [75]:
submission

,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0
...,...,...
23485,53478,0
23486,25600,0
23487,45409,0
23488,1186,0


In [71]:
from google.colab import files
files.download("final_submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>